In [1]:
from VariationalBayes import VectorParam, ScalarParam, PosDefMatrixParam, ModelParamsDict
from autograd import grad, hessian, jacobian
import math
import autograd.numpy as np
import autograd.numpy.random as npr
import copy

In [2]:
mu = VectorParam("mu", 3, lb=0, ub=10)
mu.set(np.array([1., 2., 3.]))
mu.get()
foo = mu.get_free()
mu.set_free(foo)
mu.get()

tau = ScalarParam('tau', lb=0, ub=float("inf"))
tau.set(5)
# print dir(tau)
tau.get_free()
print tau
print mu

a = np.matrix(np.random.rand(4, 4))
sigma_val = a * a.T + np.eye(4)

print sigma_val

sigma = PosDefMatrixParam('sigma', 4)
sigma.set(sigma_val)
sigma_0 = PosDefMatrixParam('sigma0', 4)
sigma_0.set_free(sigma.get_free())
print sigma_0.get() - sigma.get()

tau: 5
mu: [ 1.  2.  3.]
[[ 2.95445015  1.15420241  1.01953405  1.11669766]
 [ 1.15420241  2.0299483   0.70450234  0.92700096]
 [ 1.01953405  0.70450234  1.80972097  0.87935028]
 [ 1.11669766  0.92700096  0.87935028  2.04441947]]
[[ -4.44089210e-16  -2.22044605e-16   0.00000000e+00  -2.22044605e-16]
 [ -2.22044605e-16  -4.44089210e-16   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [ -2.22044605e-16   0.00000000e+00   0.00000000e+00   0.00000000e+00]]


In [25]:
# Build an object to contain a variational approximation to a K-dimensional multivariate normal.
K = 3
mvn_par = ModelParamsDict()

mvn_par.push_param(VectorParam('e_mu', K))
mvn_par.push_param(VectorParam('var_mu', K, lb=0))

mvn_par['e_mu'].set(np.full(K, 0.1))
mvn_par['var_mu'].set(np.full(K, 2.))

print mvn_par.names()
print mvn_par.get()
print mvn_par.get_free()

print mvn_par

['var_mu_0' 'var_mu_1' 'var_mu_2' 'e_mu_0' 'e_mu_1' 'e_mu_2']
[ 2.   2.   2.   0.1  0.1  0.1]
[ 0.69314718  0.69314718  0.69314718  0.1         0.1         0.1       ]
ModelParamsList:
	var_mu: [ 2.  2.  2.]
	e_mu: [ 0.1  0.1  0.1]


In [18]:
# Generate data

N = 100
true_mu = np.random.rand(K).T
x_cov = np.random.rand(K, K)
x_cov = 0.5 * (x_cov * x_cov.T) + np.eye(K)
# Perhaps un-pythonic, I want column vectors.
x_draws = [ np.random.multivariate_normal(true_mu, x_cov) for n in range(N) ]

In [28]:
# Log likelihood
def LogLikelihood(x_row, x_info, e_mu, e_mu_outer):
    return -0.5 * (-np.dot(e_mu, np.matmul(x_info, x_row)) + \
                   -np.dot(x_row, np.matmul(x_info, e_mu)) + \
                   np.trace(x_info * (e_mu_outer)))


def UnivariateNormalExpectedEntropy(var_mu):
    return 0.5 * np.log(var_mu)


def Elbo(x_draws, mvn_par):
    x_info = np.linalg.inv(x_cov)
    var_mu = mvn_par['var_mu'].get()
    e_mu = mvn_par['e_mu'].get()
    e_mu_outer = np.outer(e_mu, e_mu) + np.diag(var_mu)

    ll = sum([ LogLikelihood(x, x_info, e_mu, e_mu_outer) for x in x_draws ])
    entropy = sum([ UnivariateNormalExpectedEntropy(var_mu_k) for var_mu_k in var_mu])

    return ll + entropy


def ElboWrapper(free_par_vec):
    # This seems to be necessary to avoid changing the type of mvn_par.
    mvn_par_ad = copy.copy(mvn_par)
    mvn_par_ad.set_free(free_par_vec)
    return Elbo(x_draws, mvn_par)

x_row = x_draws[1]
# print x_row
# print np.matmul(x_cov, x_row)
# print np.matmul(x_cov, x_row)
# print np.inner(x_row, x_row)
# print np.inner(x_row, np.matmul(x_cov, x_row))

print mvn_par.names()
print mvn_par
print mvn_par.get()
print mvn_par.get_free()

# free_par_vec = mvn_par.get_free()
print ElboWrapper(free_par_vec)

ElboGrad = grad(ElboWrapper)
ElboHess = hessian(ElboWrapper)

print ElboGrad(free_par_vec)
print ElboHess(free_par_vec)

['var_mu_0' 'var_mu_1' 'var_mu_2' 'e_mu_0' 'e_mu_1' 'e_mu_2']
ModelParamsList:
	var_mu: Autograd ArrayNode with value [ 2.  2.  2.] and 1 tape(s)
	e_mu: Autograd ArrayNode with value [ 0.1  0.1  0.1] and 1 tape(s)
[ 2.   2.   2.   0.1  0.1  0.1]
[ 0.69314718  0.69314718  0.69314718  0.1         0.1         0.1       ]
-251.395945579
[ -77.40590731  -80.12620523 -101.01960936   59.20102508   15.01802996
  -11.06108567]
[[ -77.90590731    0.            0.            0.            0.            0.        ]
 [   0.          -80.62620523    0.            0.            0.            0.        ]
 [   0.            0.         -101.51960936    0.            0.            0.        ]
 [   0.            0.            0.          -77.90590731    0.            0.        ]
 [   0.            0.            0.            0.          -80.62620523
     0.        ]
 [   0.            0.            0.            0.            0.
  -101.51960936]]
